In [1]:
import psi4
import numpy as np
import scipy.linalg as sp
psi4.set_output_file("output.dat", True)  # setting output file
psi4.set_memory(int(5e8))
numpy_memory = 2
psi4.set_options({'basis': 'cc-pvdz', 'reference': 'uhf', 'scf_type': 'df', "e_convergence":1e-12})
class Molecule:
    def __init__(self, geom_file):
        """
        sets up the molecule object
        
        input:
        geom_file: a link to a pubchem file    
            
        note:
        This class is designed to work in an iterative HF calculation. The guess matrix needs to be
        updated asap. This will always correspond to the current fock-matrix.
        """
        if """pubchem""" in geom_file:
            self.id = psi4.geometry(geom_file)
        else:
            self.id = psi4.geometry(f"""
            {geom_file}""")
        self.id.update_geometry()
        self.wfn =  psi4.core.Wavefunction.build(self.id, psi4.core.get_global_option('basis'))
        self.basis = self.wfn.basisset()
        self.integrals = psi4.core.MintsHelper(self.basis)
        self.alpha = self.wfn.nalpha()
        self.beta = self.wfn.nbeta()
        # only works for closed shell systems
        self.guessMatrix_a = "empty"
        self.guessMatrix_b = "empty"
        
        
        #setting up the inegrals
        self.nuc_rep = self.id.nuclear_repulsion_energy()
        self.overlap = self.integrals.ao_overlap().np
        self.kin = self.integrals.ao_kinetic().np
        self.pot = self.integrals.ao_potential().np
        self.elrep = self.integrals.ao_eri().np

        # defining convergence via user interactions
        self.converge = 1e-6
        

    
    def setGuess(self, new_guess=None, spin=None):
        """
        sets the guessMatrix to a new value
        
        input:
        new_guess: numpy array that represents a new fock matrix
        spin: a string, either "alpha" or "beta"
        """
        if self.guessMatrix_a == "empty" and self.guessMatrix_b == "empty":
            self.guessMatrix_a = self.displayHamiltonian()
            self.guessMatrix_b = self.displayHamiltonian()
        else:
            assert spin == "alpha" or spin == "beta", f"{spin}: no valid spin"
            if spin == "alpha":
                self.guessMatrix_a = new_guess
            else:
                self.guessMatrix_b = new_guess


    def displayNucRep(self):
        """
        Will calculate the nuclear repulsion
        """

        return self.nuc_rep


    def displayOverlap(self):
        """
        Will display the overlap matrix as np array
        """
        return self.overlap

    def displayE_kin(self):
        """
        Will display kinetic energy as np array
        """
        return self.kin


    def displayE_pot(self):
        """
        Will display the kinetic energy as np array
        """
        return self.pot


    def displayHamiltonian(self):
        """
        Will display the hamiltonian as a np array
        """
        return self.displayE_kin() + self.displayE_pot()


    def displayElectronRepulsion(self):
        """
        Will display the interelectronic repulsion as a np array (4D array)
        """
        return self.elrep


    def transformToUnity(self):
        """
        Gives the matrix that will transform S into I_n
        
        note:
        functions return dimension objects, do not use equality
        """
        transformMatrix = self.integrals.ao_overlap()
        transformMatrix.power(-0.5, 1e-16)
        return transformMatrix.np


    def getEigenStuff(self, spin):
        """
        calculates the eigenvectors and eigenvalues of the hamiltonian
        input:
        spin: a string, either "alpha" or "beta"
        """
        if spin == "alpha":
            F = self.guessMatrix_a
        else:
            F = self.guessMatrix_b        
        return sp.eigh(F, b=self.displayOverlap())


    def getDensityMatrix(self, spin):
        """
        generates the densitiy matrices on the MO level, D_alpha, D_beta
        
        input:
        spin: a string, either "alpha" or "beta"
        """
        assert spin == "alpha" or spin == "beta", f"{spin}: no valid spin"
        if spin == "alpha":
            occ = self.alpha
            guess = self.guessMatrix_a
        else:
            occ = self.beta
            guess = self.guessMatrix_b
        C = self.getEigenStuff(spin)[1]
        if np.all(guess == self.displayHamiltonian()):
            if spin == "beta":
                k = 1
                HOMO_LUMO = C[occ-1:occ+1]
                HOMO = HOMO_LUMO[0]
                LUMO = HOMO_LUMO[1]
                HOMO_LUMO[0] += k*LUMO
                HOMO_LUMO[1] += -k*HOMO
                HOMO_LUMO *= 1/np.sqrt(2)
                
                C[occ-1:occ+1] = HOMO_LUMO
            
        
        D = np.einsum("pr, qr->pq", C[:, :occ], C[:, :occ], optimize=True)
        return D


    def displayFockMatrix(self, spin):
        """
        Will display the Fock matrix
        
        input:
        spin: a string, either "alpha" or "beta"
        """
        coulomb_a = np.einsum("nopq,pq->no", self.displayElectronRepulsion(), self.getDensityMatrix("alpha"), optimize=True)
        coulomb_b = np.einsum("nopq,pq->no", self.displayElectronRepulsion(), self.getDensityMatrix("beta"), optimize=True)
        exchange = np.einsum("npoq,pq->no", self.displayElectronRepulsion(), self.getDensityMatrix(spin), optimize=True)
        F = self.displayHamiltonian() + coulomb_a + coulomb_b - exchange
        return F


    def getElectronicEnergy(self):
        """
        calculates the energy with the current fock matrix
        """
        sumMatrix_alpha = self.displayHamiltonian() + self.guessMatrix_a
        E_alpha = 0.5*np.einsum("pq,pq->", self.getDensityMatrix("alpha"), sumMatrix_alpha, optimize=True)
        sumMatrix_beta = self.displayHamiltonian() + self.guessMatrix_b
        E_beta = 0.5*np.einsum("pq,pq->", self.getDensityMatrix("beta"), sumMatrix_beta, optimize=True)
        return E_alpha + E_beta 


    def getTotalEnergy(self):
        """
        Calculates the total energy
        """
        return self.getElectronicEnergy() + self.displayNucRep()



    def iterator(self, criterion='density', iteration=500):
        """
        Function that performs the Hartree-Fock iterative calculations for the given molecule.
        
        input:
        criterion: "energy" or "density", sets the criterion that we want to evaluate. Default "density"
        iteration: maximum amount of iterations allowed. Default 500
        
        note:
        the molecule needs to have its guessmatrices set before entering
        """
        assert self.guessMatrix_a != "empty" and self.guessMatrix_b != "empty", "make a guess first"
        assert criterion == "energy" or criterion == "density", f" {criterion}: not a valid criterion"
        # setting up entry parameters for the while loop
        E_new = 0  
        E_old = 0
        d_old_alpha = self.getDensityMatrix("alpha")
        d_old_beta = self.getDensityMatrix("beta")
        convergence = False

        # step 2: start iterating
        itercount = 0
        while not convergence and itercount < iteration:

            # calculating block: calculates energies
            E_new = self.getElectronicEnergy()
            E_total = self.getTotalEnergy()

            # generating block: generates new matrices UHF: account for alpha and beta
            F_a =  self.displayFockMatrix("alpha")
            self.setGuess(F_a, "alpha")
            F_b = self.displayFockMatrix("beta")
            self.setGuess(F_b, "beta") 
            d_new_alpha = self.getDensityMatrix("alpha")
            d_new_beta = self.getDensityMatrix("beta")

            # comparing block: will answer the "Are we there yet?" question
            rms_D_a = np.sqrt(np.einsum("pq->", (d_old_alpha - d_new_alpha)**2, optimize=True))
            rms_D_b = np.sqrt(np.einsum("pq->", (d_old_beta - d_new_beta)**2, optimize=True))
            if criterion == "density":
                if rms_D_a < self.converge and rms_D_b < self.converge:
                    convergence = True
            else:
                if abs(E_old - E_new) < self.converge:
                    convergence = True


            # maintenance block: keeps everything going
            print(f"iteration: {itercount}, E_tot: {E_total: .8f}, E_elek: {E_new: .8f}, deltaE: {E_new - E_old: .8f}, rmsD: {rms_D_a: .8f}")
            E_old = E_new
            d_old_alpha = d_new_alpha
            d_old_beta = d_new_beta
            itercount += 1
        
        return E_total

    
    def setConvergence(self, new_convergence):
        """ sets the convergence to desired value"""
        self.converge = new_convergence
    
        

In [2]:
h3 = Molecule("""
H 0 0 0
H 0 0.86602540378 0.5
H 0 0 1
units angstrom""")

In [3]:
h3.setGuess()

In [4]:
h3.setConvergence(1e-12)
h3.iterator(criterion="density", iteration=5000)

/usr/local/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:210: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
/usr/local/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:56: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


iteration: 0, E_tot: -2.24803095, E_elek: -3.83556259, deltaE: -3.83556259, rmsD:  0.98918247
iteration: 1, E_tot: -1.65600507, E_elek: -3.24353670, deltaE:  0.59202588, rmsD:  0.74968072
iteration: 2, E_tot: -1.47952400, E_elek: -3.06705563, deltaE:  0.17648107, rmsD:  0.23281459
iteration: 3, E_tot: -1.50038938, E_elek: -3.08792101, deltaE: -0.02086538, rmsD:  0.08055367
iteration: 4, E_tot: -1.50411686, E_elek: -3.09164849, deltaE: -0.00372748, rmsD:  0.03165320
iteration: 5, E_tot: -1.50485954, E_elek: -3.09239117, deltaE: -0.00074268, rmsD:  0.01364768
iteration: 6, E_tot: -1.50500901, E_elek: -3.09254064, deltaE: -0.00014947, rmsD:  0.00625144
iteration: 7, E_tot: -1.50503640, E_elek: -3.09256804, deltaE: -0.00002739, rmsD:  0.00298177
iteration: 8, E_tot: -1.50503941, E_elek: -3.09257105, deltaE: -0.00000301, rmsD:  0.00146417
iteration: 9, E_tot: -1.50503837, E_elek: -3.09257000, deltaE:  0.00000105, rmsD:  0.00073506
iteration: 10, E_tot: -1.50503720, E_elek: -3.09256883, delt

iteration: 95, E_tot: -1.50503555, E_elek: -3.09256719, deltaE: -0.00000000, rmsD:  0.00012232
iteration: 96, E_tot: -1.50503556, E_elek: -3.09256719, deltaE: -0.00000000, rmsD:  0.00012605
iteration: 97, E_tot: -1.50503556, E_elek: -3.09256719, deltaE: -0.00000000, rmsD:  0.00012990
iteration: 98, E_tot: -1.50503556, E_elek: -3.09256719, deltaE: -0.00000000, rmsD:  0.00013386
iteration: 99, E_tot: -1.50503556, E_elek: -3.09256719, deltaE: -0.00000000, rmsD:  0.00013794
iteration: 100, E_tot: -1.50503556, E_elek: -3.09256719, deltaE: -0.00000000, rmsD:  0.00014215
iteration: 101, E_tot: -1.50503556, E_elek: -3.09256719, deltaE: -0.00000000, rmsD:  0.00014648
iteration: 102, E_tot: -1.50503556, E_elek: -3.09256719, deltaE: -0.00000000, rmsD:  0.00015095
iteration: 103, E_tot: -1.50503556, E_elek: -3.09256719, deltaE: -0.00000000, rmsD:  0.00015555
iteration: 104, E_tot: -1.50503556, E_elek: -3.09256720, deltaE: -0.00000000, rmsD:  0.00016029
iteration: 105, E_tot: -1.50503556, E_elek: -

iteration: 190, E_tot: -1.50503923, E_elek: -3.09257086, deltaE: -0.00000021, rmsD:  0.00211592
iteration: 191, E_tot: -1.50503946, E_elek: -3.09257109, deltaE: -0.00000023, rmsD:  0.00218003
iteration: 192, E_tot: -1.50503970, E_elek: -3.09257133, deltaE: -0.00000024, rmsD:  0.00224606
iteration: 193, E_tot: -1.50503996, E_elek: -3.09257159, deltaE: -0.00000026, rmsD:  0.00231406
iteration: 194, E_tot: -1.50504023, E_elek: -3.09257186, deltaE: -0.00000027, rmsD:  0.00238409
iteration: 195, E_tot: -1.50504052, E_elek: -3.09257215, deltaE: -0.00000029, rmsD:  0.00245620
iteration: 196, E_tot: -1.50504083, E_elek: -3.09257246, deltaE: -0.00000031, rmsD:  0.00253046
iteration: 197, E_tot: -1.50504115, E_elek: -3.09257278, deltaE: -0.00000033, rmsD:  0.00260693
iteration: 198, E_tot: -1.50504150, E_elek: -3.09257313, deltaE: -0.00000035, rmsD:  0.00268565
iteration: 199, E_tot: -1.50504187, E_elek: -3.09257350, deltaE: -0.00000037, rmsD:  0.00276671
iteration: 200, E_tot: -1.50504225, E_el

iteration: 286, E_tot: -1.50565240, E_elek: -3.09318404, deltaE: -0.00001914, rmsD:  0.01901883
iteration: 287, E_tot: -1.50567157, E_elek: -3.09320320, deltaE: -0.00001917, rmsD:  0.01900036
iteration: 288, E_tot: -1.50569073, E_elek: -3.09322236, deltaE: -0.00001915, rmsD:  0.01896375
iteration: 289, E_tot: -1.50570983, E_elek: -3.09324146, deltaE: -0.00001911, rmsD:  0.01890911
iteration: 290, E_tot: -1.50572886, E_elek: -3.09326049, deltaE: -0.00001902, rmsD:  0.01883664
iteration: 291, E_tot: -1.50574776, E_elek: -3.09327939, deltaE: -0.00001890, rmsD:  0.01874663
iteration: 292, E_tot: -1.50576651, E_elek: -3.09329814, deltaE: -0.00001875, rmsD:  0.01863944
iteration: 293, E_tot: -1.50578507, E_elek: -3.09331670, deltaE: -0.00001856, rmsD:  0.01851552
iteration: 294, E_tot: -1.50580340, E_elek: -3.09333503, deltaE: -0.00001834, rmsD:  0.01837537
iteration: 295, E_tot: -1.50582149, E_elek: -3.09335312, deltaE: -0.00001809, rmsD:  0.01821958
iteration: 296, E_tot: -1.50583929, E_el

...

iteration: 665, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000019
iteration: 666, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000018
iteration: 667, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000018
iteration: 668, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000017
iteration: 669, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000017
iteration: 670, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000016
iteration: 671, E_tot: -1.50627432, E_elek: -3.09380595, deltaE:  0.00000000, rmsD:  0.00000015
iteration: 672, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000015
iteration: 673, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000015
iteration: 674, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000014
iteration: 675, E_tot: -1.50627432, E_el

iteration: 761, E_tot: -1.50627432, E_elek: -3.09380595, deltaE:  0.00000000, rmsD:  0.00000001
iteration: 762, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000001
iteration: 763, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000001
iteration: 764, E_tot: -1.50627432, E_elek: -3.09380595, deltaE:  0.00000000, rmsD:  0.00000001
iteration: 765, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000001
iteration: 766, E_tot: -1.50627432, E_elek: -3.09380595, deltaE:  0.00000000, rmsD:  0.00000001
iteration: 767, E_tot: -1.50627432, E_elek: -3.09380595, deltaE:  0.00000000, rmsD:  0.00000001
iteration: 768, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000001
iteration: 769, E_tot: -1.50627432, E_elek: -3.09380595, deltaE:  0.00000000, rmsD:  0.00000001
iteration: 770, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000001
iteration: 771, E_tot: -1.50627432, E_el

iteration: 856, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000000
iteration: 857, E_tot: -1.50627432, E_elek: -3.09380595, deltaE:  0.00000000, rmsD:  0.00000000
iteration: 858, E_tot: -1.50627432, E_elek: -3.09380595, deltaE:  0.00000000, rmsD:  0.00000000
iteration: 859, E_tot: -1.50627432, E_elek: -3.09380595, deltaE:  0.00000000, rmsD:  0.00000000
iteration: 860, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000000
iteration: 861, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000000
iteration: 862, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000000
iteration: 863, E_tot: -1.50627432, E_elek: -3.09380595, deltaE:  0.00000000, rmsD:  0.00000000
iteration: 864, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000000
iteration: 865, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000000
iteration: 866, E_tot: -1.50627432, E_el

iteration: 952, E_tot: -1.50627432, E_elek: -3.09380595, deltaE:  0.00000000, rmsD:  0.00000000
iteration: 953, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000000
iteration: 954, E_tot: -1.50627432, E_elek: -3.09380595, deltaE:  0.00000000, rmsD:  0.00000000
iteration: 955, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000000
iteration: 956, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000000
iteration: 957, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000000
iteration: 958, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000000
iteration: 959, E_tot: -1.50627432, E_elek: -3.09380595, deltaE:  0.00000000, rmsD:  0.00000000
iteration: 960, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000000
iteration: 961, E_tot: -1.50627432, E_elek: -3.09380595, deltaE:  0.00000000, rmsD:  0.00000000
iteration: 962, E_tot: -1.50627432, E_el

-1.5062743202993887

In [37]:
psi4.set_options({"basis": "cc-pvdz"})
m1 = Molecule("""
O
H 1 0.96
H 1 0.96 2 104.5
symmetry c1
""")
m1.setGuess()
m1.setConvergence(1e-10)
m1.iterator(iteration=5000)

/usr/local/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:210: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
/usr/local/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:56: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


iteration: 0, E_tot: -123.75569351, E_elek: -132.92388680, deltaE: -132.92388680, rmsD:  44.53246397
iteration: 1, E_tot: -70.96844194, E_elek: -80.13663524, deltaE:  52.78725156, rmsD:  35.03538947
iteration: 2, E_tot: -75.65803400, E_elek: -84.82622730, deltaE: -4.68959206, rmsD:  4.62290019
iteration: 3, E_tot: -75.89486622, E_elek: -85.06305951, deltaE: -0.23683222, rmsD:  1.70397757
iteration: 4, E_tot: -75.98347529, E_elek: -85.15166858, deltaE: -0.08860907, rmsD:  0.79474108
iteration: 5, E_tot: -76.00566271, E_elek: -85.17385601, deltaE: -0.02218742, rmsD:  0.42133207
iteration: 6, E_tot: -76.01518184, E_elek: -85.18337514, deltaE: -0.00951913, rmsD:  0.23518570
iteration: 7, E_tot: -76.02000212, E_elek: -85.18819542, deltaE: -0.00482028, rmsD:  0.13510103
iteration: 8, E_tot: -76.02267072, E_elek: -85.19086401, deltaE: -0.00266860, rmsD:  0.07938970
iteration: 9, E_tot: -76.02422515, E_elek: -85.19241845, deltaE: -0.00155443, rmsD:  0.04701505
iteration: 10, E_tot: -76.0251577

-76.02665366188079

In [6]:
psi4.energy("scf", molecule=m1.id)

-76.02663273512442

In [5]:
psi4.set_options({"basis": "cc-pvdz"})
o = Molecule("""pubchem:oxygen""")
o.setConvergence(1e-12)
o.setGuess()
o.iterator(iteration=5000)

	Searching PubChem database for oxygen (single best match returned)
	Found 1 result(s)


/usr/local/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:210: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
/usr/local/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:56: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


iteration: 0, E_tot: -249.98191282, E_elek: -277.47163805, deltaE: -277.47163805, rmsD:  3.36953456
iteration: 1, E_tot: -139.90610739, E_elek: -167.39583262, deltaE:  110.07580543, rmsD:  1.14060060
iteration: 2, E_tot: -148.98563202, E_elek: -176.47535725, deltaE: -9.07952463, rmsD:  0.28028380
iteration: 3, E_tot: -149.22778409, E_elek: -176.71750932, deltaE: -0.24215206, rmsD:  0.08876054
iteration: 4, E_tot: -149.31385295, E_elek: -176.80357818, deltaE: -0.08606887, rmsD:  0.03168964
iteration: 5, E_tot: -149.34284216, E_elek: -176.83256739, deltaE: -0.02898921, rmsD:  0.01671418
iteration: 6, E_tot: -149.35922553, E_elek: -176.84895076, deltaE: -0.01638337, rmsD:  0.01709449
iteration: 7, E_tot: -149.37671406, E_elek: -176.86643929, deltaE: -0.01748853, rmsD:  0.02154915
iteration: 8, E_tot: -149.40011796, E_elek: -176.88984319, deltaE: -0.02340390, rmsD:  0.02572350
iteration: 9, E_tot: -149.42785592, E_elek: -176.91758115, deltaE: -0.02773796, rmsD:  0.02753466
iteration: 10, E

iteration: 85, E_tot: -149.56478167, E_elek: -177.05450690, deltaE:  0.00000000, rmsD:  0.00000001
iteration: 86, E_tot: -149.56478167, E_elek: -177.05450690, deltaE:  0.00000000, rmsD:  0.00000000
iteration: 87, E_tot: -149.56478167, E_elek: -177.05450690, deltaE: -0.00000000, rmsD:  0.00000000
iteration: 88, E_tot: -149.56478167, E_elek: -177.05450690, deltaE:  0.00000000, rmsD:  0.00000000
iteration: 89, E_tot: -149.56478167, E_elek: -177.05450690, deltaE:  0.00000000, rmsD:  0.00000000
iteration: 90, E_tot: -149.56478167, E_elek: -177.05450690, deltaE:  0.00000000, rmsD:  0.00000000
iteration: 91, E_tot: -149.56478167, E_elek: -177.05450690, deltaE:  0.00000000, rmsD:  0.00000000
iteration: 92, E_tot: -149.56478167, E_elek: -177.05450690, deltaE: -0.00000000, rmsD:  0.00000000
iteration: 93, E_tot: -149.56478167, E_elek: -177.05450690, deltaE: -0.00000000, rmsD:  0.00000000
iteration: 94, E_tot: -149.56478167, E_elek: -177.05450690, deltaE:  0.00000000, rmsD:  0.00000000
iteration:

iteration: 171, E_tot: -149.56478167, E_elek: -177.05450690, deltaE: -0.00000000, rmsD:  0.00000552
iteration: 172, E_tot: -149.56478167, E_elek: -177.05450690, deltaE: -0.00000000, rmsD:  0.00000610
iteration: 173, E_tot: -149.56478167, E_elek: -177.05450690, deltaE: -0.00000000, rmsD:  0.00000673
iteration: 174, E_tot: -149.56478167, E_elek: -177.05450690, deltaE: -0.00000000, rmsD:  0.00000743
iteration: 175, E_tot: -149.56478167, E_elek: -177.05450690, deltaE: -0.00000000, rmsD:  0.00000820
iteration: 176, E_tot: -149.56478167, E_elek: -177.05450690, deltaE: -0.00000000, rmsD:  0.00000906
iteration: 177, E_tot: -149.56478167, E_elek: -177.05450690, deltaE: -0.00000000, rmsD:  0.00001000
iteration: 178, E_tot: -149.56478167, E_elek: -177.05450690, deltaE: -0.00000000, rmsD:  0.00001104
iteration: 179, E_tot: -149.56478167, E_elek: -177.05450690, deltaE: -0.00000000, rmsD:  0.00001218
iteration: 180, E_tot: -149.56478167, E_elek: -177.05450690, deltaE: -0.00000000, rmsD:  0.00001345


iteration: 256, E_tot: -149.56871559, E_elek: -177.05844082, deltaE: -0.00063745, rmsD:  0.02126153
iteration: 257, E_tot: -149.56945752, E_elek: -177.05918275, deltaE: -0.00074193, rmsD:  0.02276882
iteration: 258, E_tot: -149.57031365, E_elek: -177.06003888, deltaE: -0.00085613, rmsD:  0.02425088
iteration: 259, E_tot: -149.57129178, E_elek: -177.06101701, deltaE: -0.00097813, rmsD:  0.02567084
iteration: 260, E_tot: -149.57239671, E_elek: -177.06212194, deltaE: -0.00110494, rmsD:  0.02698714
iteration: 261, E_tot: -149.57362908, E_elek: -177.06335431, deltaE: -0.00123236, rmsD:  0.02815547
iteration: 262, E_tot: -149.57498419, E_elek: -177.06470942, deltaE: -0.00135512, rmsD:  0.02913143
iteration: 263, E_tot: -149.57645126, E_elek: -177.06617649, deltaE: -0.00146707, rmsD:  0.02987399
iteration: 264, E_tot: -149.57801302, E_elek: -177.06773825, deltaE: -0.00156176, rmsD:  0.03034906
iteration: 265, E_tot: -149.57964611, E_elek: -177.06937134, deltaE: -0.00163309, rmsD:  0.03053288


iteration: 341, E_tot: -149.59887601, E_elek: -177.08860124, deltaE: -0.00000000, rmsD:  0.00001163
iteration: 342, E_tot: -149.59887601, E_elek: -177.08860124, deltaE: -0.00000000, rmsD:  0.00001036
iteration: 343, E_tot: -149.59887602, E_elek: -177.08860124, deltaE: -0.00000000, rmsD:  0.00000923
iteration: 344, E_tot: -149.59887602, E_elek: -177.08860125, deltaE: -0.00000000, rmsD:  0.00000822
iteration: 345, E_tot: -149.59887602, E_elek: -177.08860125, deltaE: -0.00000000, rmsD:  0.00000732
iteration: 346, E_tot: -149.59887602, E_elek: -177.08860125, deltaE: -0.00000000, rmsD:  0.00000652
iteration: 347, E_tot: -149.59887602, E_elek: -177.08860125, deltaE: -0.00000000, rmsD:  0.00000581
iteration: 348, E_tot: -149.59887602, E_elek: -177.08860125, deltaE: -0.00000000, rmsD:  0.00000517
iteration: 349, E_tot: -149.59887603, E_elek: -177.08860125, deltaE: -0.00000000, rmsD:  0.00000461
iteration: 350, E_tot: -149.59887603, E_elek: -177.08860126, deltaE: -0.00000000, rmsD:  0.00000411


iteration: 427, E_tot: -149.59887603, E_elek: -177.08860126, deltaE:  0.00000000, rmsD:  0.00000000
iteration: 428, E_tot: -149.59887603, E_elek: -177.08860126, deltaE: -0.00000000, rmsD:  0.00000000
iteration: 429, E_tot: -149.59887603, E_elek: -177.08860126, deltaE:  0.00000000, rmsD:  0.00000000
iteration: 430, E_tot: -149.59887603, E_elek: -177.08860126, deltaE:  0.00000000, rmsD:  0.00000000
iteration: 431, E_tot: -149.59887603, E_elek: -177.08860126, deltaE: -0.00000000, rmsD:  0.00000000
iteration: 432, E_tot: -149.59887603, E_elek: -177.08860126, deltaE:  0.00000000, rmsD:  0.00000000
iteration: 433, E_tot: -149.59887603, E_elek: -177.08860126, deltaE:  0.00000000, rmsD:  0.00000000
iteration: 434, E_tot: -149.59887603, E_elek: -177.08860126, deltaE: -0.00000000, rmsD:  0.00000000
iteration: 435, E_tot: -149.59887603, E_elek: -177.08860126, deltaE:  0.00000000, rmsD:  0.00000000
iteration: 436, E_tot: -149.59887603, E_elek: -177.08860126, deltaE:  0.00000000, rmsD:  0.00000000


-149.59887603019735

In [11]:
psi4.energy("scf")

-149.53737176061375

In [41]:
from scipy.linalg import eigh
class ConstrainedMolecule:
    def __init__(self, geom_file, radical=False):
        """initiation method, will take in a geometry"""
        #this parameter contains all parameters and methods from the Molecule class
        self.id = Molecule(geom_file)
        self.id.setGuess()
        
    
    def setChargeDensity(self, new_matrix):
        """sets the pMatrix to a new value"""
        self.pMatrix = new_matrix
        
    
    def basischanger(self):
        """
        changes to NO basis, applies constraint, then changes back
        """
        # transform p to MO basis, where mo basis = the eigenfunctions of the f_a operator
        a = self.id.getDensityMatrix("alpha")
        b = self.id.getDensityMatrix("beta")
        f_a, f_b = self.id.displayFockMatrix("alpha"), self.id.displayFockMatrix("beta")
        p = (a+b)/2
        c = eigh(f_a, self.id.overlap)[1] # we only need the c matrix, not the eigenvalues themselves,
        
        delta = (f_b-f_a)/2
        f_cs = (f_a+f_b)/2
        # pay attention, c matrices are not unitary
        c_inv = np.linalg.inv(c) # we need the inverse for later
        p_trans = np.einsum("pq, qr, rs->ps", c_inv, p, c_inv.T, optimize=True)
        delta_trans = np.einsum("pq, qr, rs->ps", c.T, delta, c, optimize=True)
        
        
         # transform the fock matrices to NO basis
        d = eigh(p_trans)[1]
        d = d[:, ::-1] #invert all collumns
    
        d_inv = np.linalg.inv(d)
        delta_no = np.einsum("pq, qr, rs->ps", d_inv, delta_trans, d_inv.T, optimize=True)
        
        alpha = self.id.alpha 
        beta = self.id.beta
        #alter first blocks
        
        delta_no[alpha:, :beta] = 0
        delta_no[:beta, alpha:] = 0
        delta_ao = np.einsum("pq, qr, rs->ps",c_inv.T.dot(d), delta_no, d.T.dot(c_inv), optimize=True)
        
        
        return f_cs - delta_ao, f_cs + delta_ao
    
    
    def iteratinator(self):
        """adapted version of the original iterator function in the compChem.Hartree_Fock Molecule object"""
        # setting up entry parameters for the while loop
        E_new = 0  
        E_old = 0
        convergence = False

        # step 2: start iterating
        itercount = 0
        while not convergence and itercount < 5000:
            # calculating block: calculates energies
            E_new = self.id.getElectronicEnergy()
            E_total = self.id.getTotalEnergy()

            # generating block: generates new matrices UHF: account for alpha and beta, first we do the CUHF alteration
            f_a_n, f_b_n = self.basischanger()
            self.id.setGuess(f_a_n, "alpha")
            self.id.setGuess(f_b_n, "beta")

            # comparing block: will answer the "Are we there yet?" question
            if abs(E_old - E_new) < self.id.converge:
                convergence = True


            # maintenance block: keeps everything going
            #print(f"iteration: {itercount}, E_tot: {E_total: .8f}, E_elek: {E_new: .8f}, deltaE: {E_new - E_old: .8f}")
            E_old = E_new

            itercount += 1
        
        return E_total
    
    def setElectrons(self, alpha, beta):
        """sets the amount electrons of each spin"""
        self.id.alpha = alpha
        self.id.beta = beta

In [42]:
psi4.set_options({"BASIS": "cc-pvdz", 'reference': 'cuhf', "scf_type":'pk'})
h1 = ConstrainedMolecule("""
H 0 0 0
H 0 0.86602540378 0.5
H 0 0 1
units angstrom""")
h1.id.setConvergence(1e-20)
h1.iteratinator()

/usr/local/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:56: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


-1.503111863177701